![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_RoBertaForTokenClassification.ipynb)

## Import ONNX RoBertaForTokenClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `RoBertaForTokenClassification` is only available since in `Spark NLP 5.1.4` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import RoBERTa models trained/fine-tuned for token classification via `RobertaForTokenClassification` or `TFRobertaForTokenClassification`. These models are usually under `Token Classification` category and have `roberta` in their labels
- Reference: [TFRobertaForTokenClassification](https://huggingface.co/transformers/model_doc/roberta.html#tfrobertafortokenclassification)
- Some [example models](https://huggingface.co/models?filter=roberta&pipeline_tag=token-classification)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp drive/MyDrive/JSL/sparknlp/sparknlp.jar .

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.29.1`. This doesn't mean it won't work with the future releases
- Albert uses SentencePiece, so we will have to install that as well

In [3]:
!pip install -q --upgrade transformers[onnx]==4.29.1 optimum tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 28.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.7/454.7 kB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/51

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [philschmid/distilroberta-base-ner-wikiann-conll2003-3-class](https://huggingface.co/philschmid/distilroberta-base-ner-wikiann-conll2003-3-class) model from HuggingFace as an example and load it as a `ORTModelForSequenceClassification`, representing an ONNX model.

In [5]:
from optimum.onnxruntime import ORTModelForTokenClassification
import tensorflow as tf

MODEL_NAME = 'philschmid/distilroberta-base-ner-wikiann-conll2003-3-class'
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForTokenClassification.from_pretrained(MODEL_NAME, export=True)

# Save the ONNX model
ort_model.save_pretrained(ONNX_MODEL)

(…)nll2003-3-class/resolve/main/config.json:   0%|          | 0.00/962 [00:00<?, ?B/s]

Framework not specified. Using pt to export to ONNX.


pytorch_model.bin:   0%|          | 0.00/326M [00:00<?, ?B/s]

(…)class/resolve/main/tokenizer_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

(…)onll2003-3-class/resolve/main/vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

(…)onll2003-3-class/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)2003-3-class/resolve/main/tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

(…)ass/resolve/main/special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Using the export variant default. Available variants are:
	- default: The default ONNX variant.
Using framework PyTorch: 2.0.1+cu118
Overriding 1 configuration item(s)
	- use_cache -> False


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [18]:
import json

# Read the vocab JSON file
with open('{}/vocab.json'.format(ONNX_MODEL), 'r') as json_file:
    tokenizer = json.load(json_file)

# let's save the vocab as txt file
with open('{}/vocab.txt'.format(ONNX_MODEL), 'w') as keys_file:
  for item in tokenizer.keys():
    keys_file.write("%s\n" % item)

Let's have a look inside these two directories and see what we are dealing with:

In [19]:
!ls -l {ONNX_MODEL}

total 321892
drwxr-xr-x 2 root root      4096 Oct 16 22:13 assets
-rw-r--r-- 1 root root      1034 Oct 16 22:10 config.json
-rw-r--r-- 1 root root 326278966 Oct 16 22:10 model.onnx
-rw-r--r-- 1 root root       280 Oct 16 22:10 special_tokens_map.json
-rw-r--r-- 1 root root       350 Oct 16 22:10 tokenizer_config.json
-rw-r--r-- 1 root root   2108715 Oct 16 22:10 tokenizer.json
-rw-r--r-- 1 root root    798293 Oct 16 22:10 vocab.json
-rw-r--r-- 1 root root    407065 Oct 16 22:18 vocab.txt


In [9]:
!mkdir {ONNX_MODEL}/assets

- As you can see, we need to move `vocab.txt` and `merges.txt` from the tokenizer to `assets` folder which Spark NLP will look for
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [20]:
# get label2id dictionary
labels = ort_model.config.id2label
# sort the dictionary based on the id
labels = [value for key,value in sorted(labels.items(), reverse=False)]

with open(ONNX_MODEL + '/assets/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

In [10]:
!mv {ONNX_MODEL}/vocab.txt {ONNX_MODEL}/assets

In [11]:
!mv {ONNX_MODEL}/merges.txt {ONNX_MODEL}/assets

Voila! We have our `vocab.txt` and `merges.txt` inside assets directory

In [21]:
!ls -lR {ONNX_MODEL}

onnx_models/philschmid/distilroberta-base-ner-wikiann-conll2003-3-class:
total 321892
drwxr-xr-x 2 root root      4096 Oct 16 22:18 assets
-rw-r--r-- 1 root root      1034 Oct 16 22:10 config.json
-rw-r--r-- 1 root root 326278966 Oct 16 22:10 model.onnx
-rw-r--r-- 1 root root       280 Oct 16 22:10 special_tokens_map.json
-rw-r--r-- 1 root root       350 Oct 16 22:10 tokenizer_config.json
-rw-r--r-- 1 root root   2108715 Oct 16 22:10 tokenizer.json
-rw-r--r-- 1 root root    798293 Oct 16 22:10 vocab.json
-rw-r--r-- 1 root root    407065 Oct 16 22:18 vocab.txt

onnx_models/philschmid/distilroberta-base-ner-wikiann-conll2003-3-class/assets:
total 852
-rw-r--r-- 1 root root     37 Oct 16 22:18 labels.txt
-rw-r--r-- 1 root root 456318 Oct 16 22:10 merges.txt
-rw-r--r-- 1 root root 407065 Oct 16 22:12 vocab.txt


## Import and Save RoBertaForTokenClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [13]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.1.3
setup Colab for PySpark 3.2.3 and Spark NLP 5.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 537.5/537.5 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 26.4 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [14]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

print("Apache Spark version: {}".format(spark.version))

Apache Spark version: 3.2.3


- Let's use `loadSavedModel` functon in `RoBertaForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `RoBertaForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [22]:
from sparknlp.annotator import *
from sparknlp.base import *

tokenClassifier = RoBertaForTokenClassification\
  .loadSavedModel(ONNX_MODEL, spark)\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [23]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [24]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your RoBertaForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [25]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 318696
drwxr-xr-x 5 root root      4096 Oct 16 22:21 fields
drwxr-xr-x 2 root root      4096 Oct 16 22:21 metadata
-rw-r--r-- 1 root root 326328924 Oct 16 22:21 roberta_classification_onnx


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny RoBertaForTokenClassification model 😊

In [27]:
tokenClassifier_loaded = RoBertaForTokenClassification.load("./{}_spark_nlp_onnx".format(ONNX_MODEL))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

You can see what labels were used to train this model via `getClasses` function:

In [28]:
# .getClasses was introduced in spark-nlp==3.4.0
tokenClassifier_loaded.getClasses()

['B-LOC', 'I-ORG', 'I-LOC', 'I-PER', 'B-ORG', 'O', 'B-PER']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [29]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    tokenClassifier_loaded
])

# couple of simple examples
example = spark.createDataFrame([["My name is Clara and I live in Berkeley, California."], ['My name is Clara and I live in Berkeley, California.']]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "ner.result").show()

+--------------------+--------------------+
|                text|              result|
+--------------------+--------------------+
|My name is Clara ...|[O, O, O, B-PER, ...|
|My name is Clara ...|[O, O, O, B-PER, ...|
+--------------------+--------------------+



That's it! You can now go wild and use hundreds of `RoBertaForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀
